In [4]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from Functions import *

In [6]:
# Importing credentials for Strava's API
from Credentials import StravaCredentials

# Copy and paste this link in your browser and extract code -> #https://www.strava.com/oauth/authorize?client_id=99205&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
data = StravaCredentials.data

# In case you need to get the code again uncomment these:
# webbrowser.open(f"https://www.strava.com/oauth/authorize?client_id={data['client_id']}&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all")
data['code'] = input("From the web broswer enter the code:")

In [7]:
# Creating function that returns the access token that is used in the other api calls
def GetToken(data):
    token = requests.post(url= 'https://www.strava.com/api/v3/oauth/token',data=data).json()
    access_token = token['access_token']
    
    return access_token



access_token = GetToken(data)

In [10]:
# This API request gives us the general list of activities. 
# The table lacks certain details that we will get from another API request 
page = 1
url = "https://www.strava.com/api/v3/activities"
#access_token = token['access_token']
#access_token = token
# Create the dataframe ready for the API call to store your activity data

activities = pd.DataFrame()

# define a function to retrieve activities from Strava API
def retrieve_activities(access_token):
    url = "https://www.strava.com/api/v3/activities"
    activities = pd.DataFrame()
    page = 1
    while True:
        # get page of activities from Strava
        print('Getting page number:', page)
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        
        # check for rate limit exceeded error
        if r.status_code != 200:
            print('Error:',r.status_code, 'stopping extraction')
            break
    
        else:
            r = r.json()
            print(f'Extraction of page {page} complete')
            # if no results then exit loop
            if (not r):
                print('Extraction done')
                break
            r = pd.json_normalize(r)
            # adding the new table to the data frame that is storing all the data
            activities = pd.concat([activities, r])
            page += 1

    try:
        # clean up the dataframe
        clean_activities = activities[['id',
            'name',
            'distance',
            'elapsed_time',
            'total_elevation_gain',
            'sport_type',
            'start_date',
            'achievement_count',
            'athlete_count',
            'start_latlng',
            'end_latlng',
            'average_speed',
            'max_speed',
            'average_temp',
            'average_heartrate',
            'max_heartrate',
            'average_cadence',
            'elev_high',
            'elev_low']]
        return clean_activities
    except:
        ('Error occurred during extraction')

# call the function to retrieve activities
activities = retrieve_activities(access_token=access_token)
clean_activities = activities

Getting page number: 1
Extraction of page 1 complete
Getting page number: 2
Extraction of page 2 complete
Getting page number: 3
Extraction of page 3 complete
Getting page number: 4
Extraction of page 4 complete
Getting page number: 5
Extraction of page 5 complete
Getting page number: 6
Extraction of page 6 complete
Extraction done


,id,name,distance,elapsed_time,total_elevation_gain,sport_type,start_date,achievement_count,athlete_count,start_latlng,end_latlng,average_speed,max_speed,average_temp,average_heartrate,max_heartrate,average_cadence,elev_high,elev_low
0,10620155607,Morning Workout,0.0,3470,0.0,Workout,2024-01-24T12:42:09Z,0,1,[],[],0.000,0.0,30.0,146.6,179.0,NaN,0.0,0.0
1,10610368952,Afternoon Yoga,0.0,3164,0.0,Yoga,2024-01-22T22:36:36Z,0,1,[],[],0.000,0.0,33.0,99.8,142.0,NaN,0.0,0.0
2,10603329582,Lunch Workout,0.0,3597,0.0,Workout,2024-01-21T17:23:10Z,0,1,[],[],0.000,0.0,25.0,122.6,168.0,NaN,0.0,0.0
3,10597365330,Afternoon Workout,0.0,2618,0.0,Workout,2024-01-20T21:16:49Z,0,1,[],[],0.000,0.0,25.0,131.5,160.0,NaN,0.0,0.0
4,10590694930,Afternoon Workout,0.0,3341,0.0,Workout,2024-01-19T21:23:53Z,0,1,[],[],0.000,0.0,27.0,150.6,174.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,3517497833,Evening Run,3573.3,1735,4.2,Run,2020-05-26T23:20:06Z,0,1,"[27.46725, -99.505905]","[27.467154, -99.505893]",2.061,4.9,NaN,NaN,NaN,NaN,125.7,120.4
63,3516568449,Afternoon Ride,10644.4,2310,13.9,Ride,2020-05-26T19:15:34Z,0,1,"[27.484089, -99.508577]","[27.483884, -99.508626]",4.695,10.8,NaN,NaN,NaN,NaN,131.3,119.2
64,3509004014,Morning Run,4431.9,2175,15.6,Run,2020-05-25T12:58:51Z,0,1,"[27.466956, -99.506607]","[27.467437, -99.506648]",2.065,4.7,NaN,NaN,NaN,NaN,130.2,120.4
65,3496090180,Finished Thesis,3447.8,1958,3.9,Run,2020-05-23T01:51:23Z,0,1,"[27.466581, -99.505082]","[27.467184, -99.505856]",1.832,4.8,NaN,NaN,NaN,NaN,125.9,120.4


In [2]:
# Setting up parameters for gspread - updating google sheet

service_file_path = r'C:\Users\USER\Desktop\Learning\PyStrava\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'
myscope = ['https://spreadsheets.google.com/feeds', 
            'https://www.googleapis.com/auth/drive']

mycred = ServiceAccountCredentials.from_json_keyfile_name(service_file_path,myscope) # type: ignore
client = gspread.authorize(mycred)
mysheet = client.open('workout-data').sheet1
list_of_row = mysheet.get_all_records()
all_workouts_df = pd.DataFrame(list_of_row)

In [3]:
all_workouts_df = CreateScoreColumns(all_workouts_df)